In [ ]:
import numpy as np
import time

# start = time.time()


# 데이터 설정
S_i = [10450, 10450, 8272, 8272]  # 포화교통류율 - 주도로, 주도로, 부도로, 부도로
v = [8880, 6595, 2964, 1309]  # 도착 교통량 - 역삼역 방면, 선릉역 방면, 봉은사로 방면, 강남 세브란스병원 방면

C_min = 30                      # 최소 신호주기
C_max = 240                     # 최대 신호주기
num_phases = 3                  # 현시 수

# 초기 위치 설정
initial_g_i = np.random.uniform(C_min, C_max/3, num_phases)  # 녹색 신호 시간은 적당한 범위 내에서 설정
initial_L_i = np.random.uniform(3, 6, num_phases)  # 소실 시간은 3~6초 사이에서 랜덤하게 설정
initial_position = np.concatenate([initial_g_i, initial_L_i])


# 목적 함수 정의
def objective_function(params):
    g_i = params[:num_phases]
    L_i = params[num_phases:]
    C = sum(g_i) + sum(L_i)
    
    # 제약 조건 확인 및 위반 시 패널티 부여
    if C < C_min or C > C_max:
        return float('inf')
    
    if np.any(g_i > C_max - sum(L_i)):     
        return float('inf') 
    
    if np.any(L_i < 3) or np.any(L_i > 6):
        return float('inf') 

    delays = []
    capacities = []
    for i in range(num_phases):
        λ_i = g_i[i] / C
        Q_i = S_i[i] * λ_i
        capacities.append(Q_i)
        X = v[i] / Q_i
        delay = ((C - λ_i) ** 2 / (2 * (1 - λ_i * X))) + (X ** 2 / (2 * v[i] * (1 - X))) - (0.65 * (C / (v[i] ** 2)) ** (1/3) * X ** (2 + 5 * λ_i))
        delays.append(delay)
    
    total_delay = sum(delays)
    total_capacity = sum(capacities)
    
    a1 = 0.3  # 가중치 설정
    a2 = 0.4  # 가중치 설정
    return a1 * total_delay + a2 * (1 - total_capacity)


# PSO 파라미터 설정
num_particles = 3
num_iterations = 500
w = 0.5   # 관성 중량
c1 = 1.5  # 개인 학습 요인
c2 = 1.5  # 사회적 학습 요인


# PSO 알고리즘 초기화
particles = np.tile(initial_position, (num_particles, 1)).astype(float)  # 모든 입자를 초기 위치로 설정하고 float으로 변환
velocities = np.random.uniform(-0.5, 0.5, (num_particles, num_phases * 2))  # 초기 속도를 줄임
pbest_positions = particles.copy()
pbest_scores = np.array([objective_function(p) for p in particles])
gbest_position = pbest_positions[pbest_scores.argmin()]
gbest_score = pbest_scores.min()


# PSO 알고리즘 실행
for iteration in range(num_iterations):
    for i in range(num_particles):
        r1, r2 = np.random.rand(), np.random.rand()
        velocities[i] = (w * velocities[i]
                         + c1 * r1 * (pbest_positions[i] - particles[i])
                         + c2 * r2 * (gbest_position - particles[i]))
        particles[i] += velocities[i]
        
        # 제약 조건을 만족하도록 수정
        C = sum(particles[i][:num_phases]) + sum(particles[i][num_phases:])
        if C < C_min:
            scaling_factor = C_min / C
            particles[i][:num_phases] *= scaling_factor
        elif C > C_max:
            scaling_factor = C_max / C
            particles[i][:num_phases] *= scaling_factor
        
        
        particles[i] = np.clip(particles[i], 0, C_max)  # g_i와 L_i가 비현실적으로 크지 않도록 제한

        score = objective_function(particles[i])
        if score < pbest_scores[i]:
            pbest_positions[i] = particles[i]
            pbest_scores[i] = score

        if score < gbest_score:
            gbest_position = particles[i]
            gbest_score = score

# 최적 신호주기 출력
optimal_params = gbest_position
g_i = optimal_params[:num_phases]
L_i = optimal_params[num_phases:]
optimal_signal_cycle = sum(g_i) + sum(L_i)

print(f"최적 신호주기: {optimal_signal_cycle:.1f}s")
print(g_i)
print(L_i)


# end = time.time()
# print(end-start)